In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os

# Importing Deep Learning Libraries

from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Input,Dropout,GlobalAveragePooling2D,Flatten,Conv2D,BatchNormalization,Activation,MaxPooling2D
from keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
import tensorflow as tf


In [2]:
gpus = tf.config.list_logical_devices('GPU') 
stg=tf.distribute.MirroredStrategy(gpus)

In [3]:

# folder_path = "../input/ckplus/CK+48/" 
# validation_folder_path = "../input/dataset2-modified-copy-2/dataset(2) - modified - Copy 2/images/"
# folder_path = "../input/dataset2-modified-copy-2/dataset(2) - modified - Copy 2/images/" 
# folder_path = "../input/dataset-1-modified/dataset(1) - modified/images/" 
folder_path = "/kaggle/input/emotionv7/facemo/images/" 
# folder_path = "../input/face-expression-recognition-dataset/images/"
# validation_folder_path = "../input/face-expression-recognition-dataset/images/"

In [4]:
from PIL import Image
import cv2
import os

folder_dir = '/kaggle/input/emotionv7/facemo/images/train'
SIZE = 48 # Crop the image to 48x48
DOWNSAMPLE_RATIO = 4 # Downsample the image by a factor of 4

for folder in os.listdir(folder_dir):
    for file in os.listdir(os.path.join(folder_dir, folder)):
        if file.endswith("jpg"):
            image_path = os.path.join(folder_dir, folder, file)
            img = cv2.imread(image_path)
            img_resized = cv2.resize(img, (SIZE, SIZE))
            cv2.imwrite(image_path, img_resized)
        else:
            continue

In [5]:
##expression = 'happy'
##plt.style.use('dark_background')
##plt.figure(figsize= (12,12))
##for i in range(1, 10, 1):
##    plt.subplot(3,3,i)
#     img = load_img(folder_path+"train/"+expression+"/"+
  ##  img = load_img(folder_path+"train/"+expression+"/"+
    ##              os.listdir(folder_path + "train/" + expression)[i], target_size=(picture_size, picture_size))
##    plt.imshow(img)   
##plt.show()

In [6]:
batch_size  = 32
picture_size = 48
datagen_train  = ImageDataGenerator(rescale=1./255,
                                    rotation_range=.5,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    zoom_range=0.2,
                                    fill_mode='nearest')
datagen_val = ImageDataGenerator()

train_set = datagen_train.flow_from_directory(folder_path+"train",
                                              target_size = (picture_size,picture_size),
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=True)


test_set = datagen_val.flow_from_directory(folder_path+"validation",
                                              target_size = (picture_size,picture_size),
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              shuffle=True)

Found 2827 images belonging to 5 classes.
Found 797 images belonging to 5 classes.


In [7]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from keras.layers import BatchNormalization

In [8]:

from tensorflow.keras.optimizers import Adam,SGD,RMSprop


no_of_classes = 5

with stg.scope():
    pre_trained_model = VGG16(input_shape=(48,48,3), include_top=False,
                          weights="/kaggle/input/vgg16v1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")
    
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('block1_conv1')
    last_output = last_layer.output
    
    # Flatten the output layer to 1 dimension
    x = Conv2D(128, (3,3))(last_output)
    # Add a fully connected layer with 512 hidden units and ReLU activation

    vgm = Model(pre_trained_model.input, x)
    model = Sequential()
    model.add(vgm)
    #1st CNN layer
    model.add(Conv2D(64, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    #2nd CNN layer
    model.add(Conv2D(128,(5,5),padding = 'same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout (0.25))

    #3rd CNN layer
    model.add(Conv2D(512,(3,3),padding = 'same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout (0.25))

    #4th CNN layer
    model.add(Conv2D(512,(3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())

    #Fully connected 1st layer
    model.add(Dense(256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))


    # Fully connected layer 2nd layer
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))


    # Fully connected layer 3rd layer
    model.add(Dense(512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.22))

    model.add(Dense(no_of_classes, activation='softmax'))

    opt = Adam(learning_rate = 0.0001)
    model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 46, 46, 128)       75648     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        73792     
_________________________________________________________________
activation (Activation)      (None, 44, 44, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 128)       204928    
_________________________________________________________________
batch_normalization (BatchNo (None, 22, 22, 128)       5

In [9]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("./model.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks_list = [early_stopping,checkpoint,reduce_learningrate]

epochs = 64

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [10]:
history = model.fit(train_set,
                    steps_per_epoch=train_set.n//train_set.batch_size,
                    epochs=epochs,
                    validation_data = test_set,
                    validation_steps = test_set.n//test_set.batch_size,
                    callbacks=callbacks_list
                    )

Epoch 1/64
88/88 [==============================] - 20s 99ms/step - loss: 1.8526 - accuracy: 0.2701 - val_loss: 4434.6230 - val_accuracy: 0.2174
Epoch 2/64
88/88 [==============================] - 6s 71ms/step - loss: 1.5732 - accuracy: 0.3624 - val_loss: 3001.6887 - val_accuracy: 0.1107
Epoch 3/64
88/88 [==============================] - 6s 70ms/step - loss: 1.4041 - accuracy: 0.4190 - val_loss: 1810.0820 - val_accuracy: 0.2227
Epoch 4/64
88/88 [==============================] - 6s 74ms/step - loss: 1.1599 - accuracy: 0.5388 - val_loss: 2441.3411 - val_accuracy: 0.1107
Epoch 5/64
88/88 [==============================] - 6s 73ms/step - loss: 0.8994 - accuracy: 0.6397 - val_loss: 750.0030 - val_accuracy: 0.2943
Epoch 6/64
88/88 [==============================] - 6s 71ms/step - loss: 0.6933 - accuracy: 0.7395 - val_loss: 1883.2427 - val_accuracy: 0.3307
Epoch 7/64
88/88 [==============================] - 6s 71ms/step - loss: 0.6103 - accuracy: 0.7667 - val_loss: 2598.9834 - val_accuracy:

In [11]:
test_loss, test_acc = model.evaluate(test_set)
print('Test accuracy:', test_acc)

25/25 [==============================] - 1s 34ms/step - loss: 745.4232 - accuracy: 0.2911
Test accuracy: 0.29109159111976624


In [12]:
model.save('/kaggle/working/my_model_emo.h5')